# Plot Helikite datasets

v. 2.1 - 10 October 2025

Michael Lonardi | michael.lonardi@epfl.ch

Extreme Environments Research Laboratory, École Polytechnique fédérale de Lausanne, Sion, 1951, Switzerland


Code provided as a supplement for the visualization of the data published in "Vertical observations of aerosol properties in the Arctic, Antarctic and Alpine atmospheric boundary layers with a tethered balloon" by Schmale et al., submitted to ESSD

# Sections:

0) Campaign selection:
    - Enter the path and the name of the input file 
    - Enter the path of the output directory 
1) Profiles
    - Provides vertical profiles of all the variables for each flight. Each flight is divided in ascent and descent.
2) Time series
    - Provides time series of all the variables for each flight. A dashed red line separates ascent and descent.
3) Average profile for the entire campaign
    - Same as (1) but each variable is averaged to get one line
4) Overview of the entire campaign
    - Cumulatively plots the variables of (1) into one figure

# Campaign selection

In [ ]:
import pandas as pd

# Select to read in the level 2 data:
df = pd.read_csv(r'C:\Users\yourfolder\campaignfile.csv')

# Select the output folder:
output_dir = fr"C:\Users\yourfolder"

In [ ]:
# Convert datetime column to datetime type
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Profiles

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Function to split ascent and descent
def split_ascent_descent(flight_data):
    # Find the maximum altitude point
    max_alt_idx = flight_data['Altitude'].idxmax()
    
    # Split data
    ascent = flight_data[flight_data.index <= max_alt_idx].copy()
    descent = flight_data[flight_data.index > max_alt_idx].copy()
    
    return ascent, descent

# Function to create time points every 5 minutes
def create_time_points(flight_data):
    start_time = flight_data['DateTime'].min()
    end_time = flight_data['DateTime'].max()
    
    # Create points every 5 minutes
    time_points = []
    current_time = start_time
    while current_time <= end_time:
        time_points.append(current_time)
        current_time += timedelta(minutes=5)
    
    # Always include the last point if it's not already there
    if time_points[-1] != end_time:
        time_points.append(end_time)
    
    return time_points

def add_altitude_shading(ax, alt_data, flag_cloud=None, flag_hovering=None):
    # Get current x-axis limits for shading
    xlim = ax.get_xlim()
    
    # Cloud shading (gray)
    if flag_cloud is not None and not pd.Series(flag_cloud).isna().all():
        cloud_regions = pd.Series(flag_cloud) == 1
        if cloud_regions.any():
            ax.fill_betweenx(alt_data, xlim[0], xlim[1], 
                             where=cloud_regions, color='gray', alpha=0.2, label='Cloud')
    
    # Hovering shading (orange hatched)
    if flag_hovering is not None and not pd.Series(flag_hovering).isna().all():
        hover_regions = pd.Series(flag_hovering) == 1
        if hover_regions.any():
            ax.fill_betweenx(alt_data, xlim[0], xlim[1], 
                             where=hover_regions, color='orange', alpha=0.2, hatch='///', label='Hovering')

# Get unique flights
flights = df['flight_nr'].unique()

for flight in flights:
    print(f"Processing flight {flight} of {len(flights)}")
    
    # Filter data for current flight
    flight_data = df[df['flight_nr'] == flight].copy()
    flight_data = flight_data.sort_values('DateTime')
    
    # Skip if no altitude data
    if 'Altitude' not in flight_data.columns or flight_data['Altitude'].isna().all():
        print(f"No altitude data for flight {flight}, skipping...")
        continue
    
    # Get flight date for title
    flight_date = flight_data['DateTime'].dt.date.iloc[0]
    
    # Split into ascent and descent
    ascent_data, descent_data = split_ascent_descent(flight_data)
    
    # Create figure with subplots (2 rows, 8 columns)
    fig, axes = plt.subplots(2, 8, figsize=(24, 12))
    fig.suptitle(f'Flight {flight} - {flight_date} - Vertical Profiles (Level 2)', fontsize=16, fontweight='bold')
    
    # Adjust layout
    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    
    # Set y-axis limits for all plots of a campaign
    max_alt = df['Altitude'].max()
    ylim_max = int(np.ceil(max_alt / 100.0) * 100)
    ylim = [0, ylim_max]
    
    # Row 0: Ascent, Row 1: Descent
    for row, (phase_data, phase_name) in enumerate([(ascent_data, 'Ascent'), (descent_data, 'Descent')]):
        if phase_data.empty:
            # Fill row with empty plots
            for col in range(8):
                axes[row, col].text(0.5, 0.5, f'No {phase_name} Data', 
                                  transform=axes[row, col].transAxes, ha='center', va='center')
                axes[row, col].set_ylim(ylim)
            continue
        
        # Panel 1: Time vs Altitude (every 5 minutes)
        ax = axes[row, 0]
        
        # Filter out NaT values for interpolation
        valid_time_mask = phase_data['DateTime'].notna() & phase_data['Altitude'].notna()
        valid_phase_data = phase_data[valid_time_mask].copy()
        
        if not valid_phase_data.empty:
            time_points = create_time_points(valid_phase_data)
            
            # Interpolate altitude values at time points
            alt_interp = np.interp([t.timestamp() for t in time_points], 
                                  [t.timestamp() for t in valid_phase_data['DateTime']], 
                                  valid_phase_data['Altitude'])
            
            ax.plot([mdates.date2num(t) for t in time_points], alt_interp, 'ko-', markersize=4)
        else:
            ax.text(0.5, 0.5, 'No valid time data', 
                   transform=ax.transAxes, ha='center', va='center')
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - Time Profile')
        ax.set_ylim(ylim)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        ax.tick_params(axis='x', rotation=45)

        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 2: Temperature vs Altitude
        ax = axes[row, 1]
        if 'Temperature' in phase_data.columns and not phase_data['Temperature'].isna().all():
            ax.plot(phase_data['Temperature'], phase_data['Altitude'], 'r-', label='Temperature')
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        ax.set_xlabel('Temperature (°C)')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - Temperature')
        ax.set_ylim(ylim)
        
        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                     flag_cloud=valid_phase_data.get('flag_cloud'),
                     flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 3: Relative Humidity vs Altitude
        ax = axes[row, 2]
        if 'RH' in phase_data.columns and not phase_data['RH'].isna().all():
            ax.plot(phase_data['RH'], phase_data['Altitude'], 'b-', label='RH')
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        ax.set_xlabel('Relative Humidity (%)')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - Humidity')
        ax.set_ylim(ylim)
        
        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                     flag_cloud=valid_phase_data.get('flag_cloud'),
                     flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 4: Wind Speed & Direction vs Altitude
        ax = axes[row, 3]
        ax_right = ax.twiny()
        
        has_wind_speed = 'WindSpeed' in phase_data.columns and not phase_data['WindSpeed'].isna().all()
        has_wind_dir = 'WindDir' in phase_data.columns and not phase_data['WindDir'].isna().all()
        
        if has_wind_speed: 
            ax.plot(phase_data['WindSpeed'], phase_data['Altitude'], 'r-', label='Wind Speed')
            ax.grid(True, alpha=0.3)
        
        if has_wind_dir:
            ax_right.scatter(phase_data['WindDir'], phase_data['Altitude'], c='green', s=20, alpha=0.3, label='Wind Dir')
            ax_right.set_xlim(0, 360)
        
        if not has_wind_speed and not has_wind_dir:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        
        ax.set_xlabel('Wind Speed (m/s)', color='red')
        ax.tick_params(axis='x', labelcolor='red')
        ax_right.set_xlabel('Wind Direction (°)', color='green')
        ax_right.tick_params(axis='x', labelcolor='green')
        
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - Wind')
        ax.set_ylim(ylim)

        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))        
        
        # Panel 5: Particle Counters vs Altitude
        ax = axes[row, 4]
        
        has_data = False
        
        if 'Partector_total_N' in phase_data.columns and not phase_data['Partector_total_N'].isna().all():
            ax.plot(phase_data['Partector_total_N'], phase_data['Altitude'], 'b-', alpha=0.3, label='Partector')
            has_data = True
        
        if 'CPC_total_N' in phase_data.columns and not phase_data['CPC_total_N'].isna().all():
            ax.plot(phase_data['CPC_total_N'], phase_data['Altitude'], 'g-', alpha=1, label='CPC')
            has_data = True
        
        if 'mSEMS_total_N' in phase_data.columns and not phase_data['mSEMS_total_N'].isna().all():
            valid_msems = phase_data.dropna(subset=['mSEMS_total_N'])
            if not valid_msems.empty:
                ax.scatter(valid_msems['mSEMS_total_N'], valid_msems['Altitude'], 
                          c='red', s=20, alpha=1, label='mSEMS')
                has_data = True
        
        if not has_data:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        else:
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        
        ax.set_xlabel('Total N (#/cm³)')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - Particles')
        ax.set_ylim(ylim)

        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 6: POPS and miniCDA Total vs Altitude
        ax = axes[row, 5]

        has_data = False

        if 'POPS_total_N' in phase_data.columns and not phase_data['POPS_total_N'].isna().all():
            ax.plot(phase_data['POPS_total_N'], phase_data['Altitude'], color='purple', label='POPS', linewidth=2)
            has_data = True

        if 'mCDA_total_N' in phase_data.columns and not phase_data['mCDA_total_N'].isna().all():
            ax.plot(phase_data['mCDA_total_N'], phase_data['Altitude'], color='orange', label='miniCDA', linewidth=2)
            has_data = True

        if not has_data:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        else:
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)

        ax.set_xlabel('Total N (#/cm³)')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - POPS & miniCDA')
        ax.set_ylim(ylim)

        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 7: CO2 vs Altitude
        ax = axes[row, 6]
        
        if 'CO2' in phase_data.columns and not phase_data['CO2'].isna().all():
            ax.plot(phase_data['CO2'], phase_data['Altitude'], color='brown', label='CO2')
            ax.grid(True, alpha=0.3)
        else:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
            
        ax.set_xlabel('CO2 (ppm)')
        ax.set_ylabel('Altitude (m)')
        ax.set_title(f'{phase_name} - CO2')
        ax.set_ylim(ylim)
       
        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))
        
        # Panel 8: STAP Sigma Values or Microaethalometer Absorption Coefficients vs Altitude
        ax = axes[row, 7]
        
        has_data = False
        
        # Check for microaethalometer data 
        microaeth_vars = ['MA_Abs_Coeff_375', 'MA_Abs_Coeff_450', 'MA_Abs_Coeff_528', 'MA_Abs_Coeff_625', 'MA_Abs_Coeff_880']
        microaeth_colors = ['purple', 'blue', 'green', 'red', 'black']
        microaeth_labels = ['375nm', '450nm', '528nm', '625nm', '880nm']
        
        has_microaeth = any(var in phase_data.columns and not phase_data[var].isna().all() for var in microaeth_vars)
        
        if has_microaeth:
            has_data = True
            # Plot microaethalometer data
            for var, color, label in zip(microaeth_vars, microaeth_colors, microaeth_labels):
                if var in phase_data.columns and not phase_data[var].isna().all():
                    ax.scatter(phase_data[var], phase_data['Altitude'], color=color, label=label, alpha=0.8)
            
            ax.set_xlabel('Absorption Coefficient')
            ax.set_title(f'{phase_name} - Microaethalometer')
        
        # Check for STAP data 
        stap_vars = ['STAP_sigmab_smth', 'STAP_sigmag_smth', 'STAP_sigmar_smth']
        stap_colors = ['blue', 'green', 'red']
        stap_labels = ['σ_blue', 'σ_green', 'σ_red']

        has_stap = any(var in phase_data.columns and not phase_data[var].isna().all() for var in stap_vars)
        
        if has_stap:        
            has_data = True
            # Plot STAP data            
            for var, color, label in zip(stap_vars, stap_colors, stap_labels):
                if var in phase_data.columns and not phase_data[var].isna().all():
                    ax.scatter(phase_data[var], phase_data['Altitude'], color=color, label=label, alpha=0.8)
            
            ax.set_xlabel('STAP σ')
            ax.set_title(f'{phase_name} - STAP')
        
        if not has_data:
            ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        else:
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        
        ax.set_ylabel('Altitude (m)')
        ax.set_ylim(ylim)
        
        add_altitude_shading(ax, valid_phase_data['Altitude'], 
                             flag_cloud=valid_phase_data.get('flag_cloud'),
                             flag_hovering=valid_phase_data.get('flag_hovering'))        
    
    plt.tight_layout()

    # Save the figure
    output_file = f"Flight_{flight}_{flight_date}_Profiles_Level2.png"
    output_filename = os.path.join(output_dir, output_file)

    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    print(f"Saved figure: {output_filename}")

    plt.show()
    plt.close(fig)
    
    print(f"Completed flight {flight} of {len(flights)}")
    
print("All flights processed!")

# Time series

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Remember to select the campaign if running only this part

# Function to find descent start time
def find_descent_start_time(flight_data):
    """Find the time when descent starts (max altitude point)"""
    max_alt_idx = flight_data['Altitude'].idxmax()
    return flight_data.loc[max_alt_idx, 'DateTime']

def add_time_shading(ax, time_data, flag_cloud=None, flag_hovering=None, flag_pollution=None):
    """Add shading to time series plots based on flags"""
    ylim = ax.get_ylim()
    
    if flag_cloud is not None and not pd.Series(flag_cloud).isna().all():
        cloud_regions = pd.Series(flag_cloud) == 1
        if cloud_regions.any():
            ax.fill_between(time_data, ylim[0], ylim[1], 
                           where=cloud_regions, color='gray', alpha=0.2, label='Cloud')
    
    if flag_hovering is not None and not pd.Series(flag_hovering).isna().all():
        hover_regions = pd.Series(flag_hovering) == 1
        if hover_regions.any():
            ax.fill_between(time_data, ylim[0], ylim[1], 
                           where=hover_regions, color='orange', alpha=0.2, label='Hovering')
    
    if flag_pollution is not None and not pd.Series(flag_pollution).isna().all():
        pollution_regions = pd.Series(flag_pollution) == 1
        if pollution_regions.any():
            ax.fill_between(time_data, ylim[0], ylim[1], 
                           where=pollution_regions, color='purple', alpha=0.2, label='Pollution')

def plot_heatmap_bins(ax, time_data, bin_data, bin_prefix, title_name, descent_time, time_extent):
    """Create heatmap for size distribution bins"""
    # Find all bin columns
    bin_cols = [col for col in bin_data.columns if col.startswith(bin_prefix)]
    
    if not bin_cols:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        ax.set_xlim(time_extent)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
        ax.set_title(title_name)
        ax.set_ylabel('Bin number')
        return
    
    bin_matrix_data = bin_data[bin_cols].copy()
    valid_rows = ~bin_matrix_data.isna().all(axis=1)
    
    if not valid_rows.any():
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
        ax.set_xlim(time_extent)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
        ax.set_title(title_name)
        ax.set_ylabel('Bin number')
        return
    
    valid_bin_data = bin_matrix_data[valid_rows]
    valid_time_data = time_data[valid_rows]
    bin_matrix = valid_bin_data.values.T
    bin_matrix = np.where(np.isnan(bin_matrix), 0.1, bin_matrix)
    bin_matrix = np.where(bin_matrix <= 0, 0.1, bin_matrix)
    
    if bin_matrix.size > 0 and np.max(bin_matrix) > 0.1:
        time_num = mdates.date2num(valid_time_data)
        if len(time_num) > 1:
            dt = (time_num[-1] - time_num[0]) / (len(time_num) - 1)
            time_edges = np.concatenate([time_num - dt/2, [time_num[-1] + dt/2]])
        else:
            time_edges = np.array([time_num[0] - 0.5, time_num[0] + 0.5])
        
        bin_edges = np.arange(len(bin_cols) + 1)
        T_edges, B_edges = np.meshgrid(time_edges, bin_edges)
        
        im = ax.pcolormesh(T_edges, B_edges, bin_matrix, cmap='viridis', shading='flat')
        
        fig = ax.get_figure()
        cbar_ax = fig.add_axes([
            ax.get_position().x1 + 0.07, 
            ax.get_position().y0 - 0.01,
            0.015,
            ax.get_position().height
        ])

        cbar = fig.colorbar(im, cax=cbar_ax)
        cbar.set_label('Concentration')
        
        tick_positions = np.arange(0, len(bin_cols), max(1, len(bin_cols)//10))
        ax.set_yticks(tick_positions)
        ax.set_yticklabels([f'Bin {int(i)+1}' for i in tick_positions])
    else:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    
    if descent_time is not None:
        ax.axvline(x=descent_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    ax.set_xlim(time_extent)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_title(title_name)
    ax.set_ylabel('Bin number')
    ax.grid(True, alpha=0.3)

# Define bins centers (Might need to be adapted following campaign metadata)
msems_diameters = np.array([
    8.2, 8.7, 9.1, 9.6, 10.2, 10.7, 11.3, 11.9, 12.5, 13.2,
    14.0, 14.7, 15.5, 16.4, 17.3, 18.2, 19.2, 20.3, 21.4, 22.6,
    23.8, 25.1, 26.5, 28.0, 29.6, 31.2, 33.0, 34.8, 36.8, 38.9,
    41.1, 43.4, 45.9, 48.5, 51.3, 54.3, 57.5, 60.8, 64.4, 68.2,
    72.3, 76.6, 81.2, 86.1, 91.4, 97.0, 103.1, 109.6, 116.5, 124.0,
    132.0, 140.7, 150.0, 160.1, 171.0, 182.8, 195.6, 209.6, 224.7, 241.2
])

pops_diameters = [
    195.2873341, 
    212.890625, 234.121875, 272.2136986, 322.6106374, 
    422.0817873, 561.8906456, 748.8896681, 1054.138693,
    1358.502538, 1802.347716, 2440.99162, 3061.590212
]

mcda_diameters = np.array([
    0.244381, 0.246646, 0.248908, 0.251144, 0.253398, 0.255593, 0.257846, 0.260141, 0.262561, 0.265062, 0.267712, 0.270370, 0.273159, 0.275904, 0.278724, 0.281554, 0.284585, 0.287661, 0.290892, 0.294127, 0.297512, 0.300813, 0.304101, 0.307439,
    0.310919, 0.314493, 0.318336, 0.322265, 0.326283, 0.330307, 0.334409, 0.338478, 0.342743, 0.347102, 0.351648, 0.356225, 0.360972, 0.365856, 0.371028, 0.376344, 0.382058, 0.387995, 0.394223, 0.400632, 0.407341, 0.414345, 0.421740, 0.429371,
    0.437556, 0.446036, 0.454738, 0.463515, 0.472572, 0.481728, 0.491201, 0.500739, 0.510645, 0.520720, 0.530938, 0.541128, 0.551563, 0.562058, 0.572951, 0.583736, 0.594907, 0.606101, 0.617542, 0.628738, 0.640375, 0.652197, 0.664789, 0.677657,
    0.691517, 0.705944, 0.721263, 0.736906, 0.753552, 0.770735, 0.789397, 0.808690, 0.829510, 0.851216, 0.874296, 0.897757, 0.922457, 0.948074, 0.975372, 1.003264, 1.033206, 1.064365, 1.097090, 1.130405, 1.165455, 1.201346, 1.239589, 1.278023,
    1.318937, 1.360743, 1.403723, 1.446000, 1.489565, 1.532676, 1.577436, 1.621533, 1.667088, 1.712520, 1.758571, 1.802912, 1.847836, 1.891948, 1.937088, 1.981087, 2.027604, 2.074306, 2.121821, 2.168489, 2.216644, 2.263724, 2.312591, 2.361099,
    2.412220, 2.464198, 2.518098, 2.571786, 2.628213, 2.685162, 2.745035, 2.805450, 2.869842, 2.935997, 3.005175, 3.074905, 3.148598, 3.224051, 3.305016, 3.387588, 3.476382, 3.568195, 3.664863, 3.761628, 3.863183, 3.965651, 4.072830, 4.179050,
    4.289743, 4.400463, 4.512449, 4.621025, 4.731530, 4.839920, 4.949855, 5.057777, 5.169742, 5.281416, 5.395039, 5.506828, 5.621488, 5.734391, 5.849553, 5.962881, 6.081516, 6.200801, 6.322133, 6.441786, 6.565130, 6.686935, 6.813017, 6.938981,
    7.071558, 7.205968, 7.345185, 7.483423, 7.628105, 7.774385, 7.926945, 8.080500, 8.247832, 8.419585, 8.598929, 8.780634, 8.973158, 9.167022, 9.372760, 9.582145, 9.808045, 10.041607, 10.287848, 10.537226, 10.801172, 11.068405, 11.345135,
    11.621413, 11.910639, 12.200227, 12.492929, 12.780176, 13.072476, 13.359067, 13.651163, 13.937329, 14.232032, 14.523919, 14.819204, 15.106612, 15.402110, 15.695489, 15.998035, 16.297519, 16.610927, 16.926800, 17.250511,
    17.570901, 17.904338, 18.239874, 18.588605, 18.938763, 19.311505, 19.693678, 20.093464, 20.498208, 20.927653, 21.366609, 21.827923, 22.297936, 22.802929, 23.325426, 23.872344, 24.428708, 25.016547, 25.616663, 26.249815,
    26.888493, 27.563838, 28.246317, 28.944507, 29.626186, 30.323440, 31.005915, 31.691752, 32.353900, 33.030123, 33.692286, 34.350532, 34.984611, 35.626553, 36.250913, 36.878655, 37.489663, 38.121550, 38.748073, 39.384594, 
    40.008540, 40.654627, 41.292757, 41.937789, 42.578436
])
    
# Get unique flights
flights = df['flight_nr'].unique()

for flight in flights:
    print(f"Processing flight {flight}")
    
    # Filter data for current flight
    flight_data = df[df['flight_nr'] == flight].copy()
    flight_data = flight_data.sort_values('DateTime')
    
    # DEBUG: Check available columns and data
    print(f"  Total columns: {len(flight_data.columns)}")
    
    # Check mSEMS columns
    msems_cols = [col for col in flight_data.columns if col.startswith('mSEMS_Bin_Conc')]
    print(f"  mSEMS columns found: {len(msems_cols)}")
    if msems_cols:
        msems_data_count = flight_data[msems_cols].count().sum()
        print(f"  mSEMS non-null values: {msems_data_count}")
        print(f"  Sample mSEMS columns: {msems_cols[:5]}")
    
    # Check POPS columns  
    pops_cols = [col for col in flight_data.columns if col.startswith('POPS_b')]
    print(f"  POPS columns found: {len(pops_cols)}")
    if pops_cols:
        pops_data_count = flight_data[pops_cols].count().sum()
        print(f"  POPS non-null values: {pops_data_count}")
        print(f"  Sample POPS columns: {pops_cols[:5]}")
    
    # Check mCDA columns
    mcda_cols = [col for col in flight_data.columns if col.startswith('mCDA_dataB')]
    print(f"  mCDA columns found: {len(mcda_cols)}")
    if mcda_cols:
        mcda_data_count = flight_data[mcda_cols].count().sum()
        print(f"  mCDA non-null values: {mcda_data_count}")
        print(f"  Sample mCDA columns: {mcda_cols[:5]}")
    
    print() # Empty line for readability
    
    # Skip if no altitude data
    if 'Altitude' not in flight_data.columns or flight_data['Altitude'].isna().all():
        print(f"No altitude data for flight {flight}, skipping...")
        continue
    
    # Get flight date for title
    flight_date = flight_data['DateTime'].dt.date.iloc[0]
    
    # Find descent start time
    descent_start_time = find_descent_start_time(flight_data)
    
    # Define global time extent for this flight
    time_extent = [flight_data['DateTime'].min(), flight_data['DateTime'].max()]
    
    # Create figure with subplots (10 rows, 1 column)
    fig, axes = plt.subplots(10, 1, figsize=(16, 22))
    fig.suptitle(f'Flight {flight} - {flight_date} - Time Series', fontsize=16, fontweight='bold')
    
    # Adjust layout
    plt.subplots_adjust(hspace=0.4)
    
    time_data = flight_data['DateTime']
    
    # Panel 0: Altitude
    ax = axes[0]
    if not flight_data['Altitude'].isna().all():
        ax.plot(time_data, flight_data['Altitude'], 'k-', linewidth=2)
    
    # Add descent start line
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8, label='Descent start')
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    ax.set_title('Altitude')
    ax.set_ylabel('Altitude (m)')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 1: Temperature/RH with dual y-axis
    ax = axes[1]
    ax_right = ax.twinx()
    
    has_temp = 'Temperature' in flight_data.columns and not flight_data['Temperature'].isna().all()
    has_rh = 'RH' in flight_data.columns and not flight_data['RH'].isna().all()
    
    # Temperature
    if has_temp:
        ax.plot(time_data, flight_data['Temperature'], 'r-', label='Temperature', linewidth=2)
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    if not has_temp and not has_rh:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    
    ax.set_title('Temperature/RH')
    ax.set_ylabel('Temperature (°C)', color='red')
    ax.tick_params(axis='y', labelcolor='red')
    
    # Relative Humidity
    if has_rh:
        ax_right.plot(time_data, flight_data['RH'], 'b-', label='RH', linewidth=2)
        ax_right.set_ylim(0, 100)
    
    ax_right.set_ylabel('RH (%)', color='blue')
    ax_right.tick_params(axis='y', labelcolor='blue')
    
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 2: Wind Speed/Direction with dual y-axis
    ax = axes[2]
    ax_right = ax.twinx()
    
    has_wind_speed = 'WindSpeed' in flight_data.columns and not flight_data['WindSpeed'].isna().all()
    has_wind_dir = 'WindDir' in flight_data.columns and not flight_data['WindDir'].isna().all()
    
    # Wind Speed
    if has_wind_speed:
        ax.plot(time_data, flight_data['WindSpeed'], 'r-', label='Wind Speed', linewidth=2)
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    if not has_wind_speed and not has_wind_dir:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    
    ax.set_title('Wind Speed/Direction')
    ax.set_ylabel('Wind Speed (m/s)', color='red')
    ax.tick_params(axis='y', labelcolor='red')
    
    # Wind Direction
    if has_wind_dir:
        ax_right.scatter(time_data, flight_data['WindDir'], c='green', s=10, alpha=0.6, label='Wind Dir')
        ax_right.set_ylim(0, 360)
    
    ax_right.set_ylabel('Wind Direction (°)', color='green')
    ax_right.tick_params(axis='y', labelcolor='green')
    
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 3: Total Concentrations (Partector, CPC, mSEMS only)
    ax = axes[3]
    
    has_data = False
    
    if 'Partector_total_N' in flight_data.columns and not flight_data['Partector_total_N'].isna().all():
        ax.plot(time_data, flight_data['Partector_total_N'], color='blue', label='Partector', linewidth=2, alpha=0.7)
        has_data = True
    
    if 'CPC_total_N' in flight_data.columns and not flight_data['CPC_total_N'].isna().all():
        ax.plot(time_data, flight_data['CPC_total_N'], color='green', label='CPC', linewidth=2)
        has_data = True
    
    if 'mSEMS_total_N' in flight_data.columns and not flight_data['mSEMS_total_N'].isna().all():
        valid_msems = flight_data.dropna(subset=['mSEMS_total_N'])
        if not valid_msems.empty:
            ax.scatter(valid_msems['DateTime'], valid_msems['mSEMS_total_N'], 
                      c='red', s=15, alpha=0.8, label='mSEMS')
            has_data = True
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    if not has_data:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    else:
        ax.legend(fontsize=8)
    
    ax.set_title('Total Concentrations (Partector, CPC, mSEMS)')
    ax.set_ylabel('Total N (#/cm³)')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 4: POPS and mCDA Total Concentrations
    ax = axes[4]
    
    has_data = False
    
    if 'POPS_total_N' in flight_data.columns and not flight_data['POPS_total_N'].isna().all():
        ax.plot(time_data, flight_data['POPS_total_N'], color='purple', label='POPS', linewidth=2)
        has_data = True
    
    if 'mCDA_total_N' in flight_data.columns and not flight_data['mCDA_total_N'].isna().all():
        ax.plot(time_data, flight_data['mCDA_total_N'], color='orange', label='miniCDA', linewidth=2)
        has_data = True
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    if not has_data:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    else:
        ax.legend(fontsize=8)
    
    ax.set_title('POPS & miniCDA Total Concentrations')
    ax.set_ylabel('Total N (#/cm³)')
    ax.grid(True, alpha=0.3)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 5: mSEMS bins heatmap
    ax = axes[5]
    plot_heatmap_bins(ax, time_data, flight_data, 'mSEMS_Bin_Conc', 'mSEMS Size Distribution', descent_start_time, time_extent)
        
    # Labels
    step = 5  # thin labels so they don't overlap
    ax.set_yticks(np.arange(0, len(msems_diameters), step))
    ax.set_yticklabels([f"{d:.2f}" for d in msems_diameters[::step]], ha='right')
    ax.set_ylabel("Diameter [µm]")
    
    # Panel 6: POPS bins heatmap
    ax = axes[6]
    plot_heatmap_bins(ax, time_data, flight_data, 'POPS_b', 'POPS Size Distribution', descent_start_time, time_extent)
    
    # Labels
    ax.set_yticks(np.arange(len(pops_diameters)))
    ax.set_yticklabels([f"{d:.0f}" for d in pops_diameters], ha='right')
    ax.set_ylabel("Diameter [nm]")
    
    # Panel 7: mCDA bins heatmap
    ax = axes[7]
    plot_heatmap_bins(ax, time_data, flight_data, 'mCDA_dataB', 'mCDA Size Distribution', descent_start_time, time_extent)
    
    # Labels
    step = 20  # thin labels so they don't overlap
    ax.set_yticks(np.arange(0, len(mcda_diameters), step))
    ax.set_yticklabels([f"{d:.2f}" for d in mcda_diameters[::step]], ha='right')
    ax.set_ylabel("Diameter [µm]")

    # Panel 8: CO2
    ax = axes[8]
    if 'CO2' in flight_data.columns and not flight_data['CO2'].isna().all():
        ax.plot(time_data, flight_data['CO2'], color='brown', linewidth=2)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    ax.set_title('CO2')
    ax.set_ylabel('CO2 (ppm)')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)
    
    # Panel 9: Microaethalometer and STAP
    ax = axes[9]
    
    has_data = False
    
    # Microaethalometer data with circle markers
    microaeth_vars = ['MA_Abs_Coeff_375', 'MA_Abs_Coeff_450', 'MA_Abs_Coeff_528', 'MA_Abs_Coeff_625', 'MA_Abs_Coeff_880']
    microaeth_colors = ['purple', 'blue', 'green', 'red', 'black']
    microaeth_labels = ['Abs_375nm', 'Abs_450nm', 'Abs_528nm', 'Abs_625nm', 'Abs_880nm']
    
    for var, color, label in zip(microaeth_vars, microaeth_colors, microaeth_labels):
        if var in flight_data.columns and not flight_data[var].isna().all():
            ax.scatter(time_data, flight_data[var], color=color, label=label, alpha=0.7, s=15, marker='o')
            has_data = True
    
    # STAP data with square markers
    stap_vars = ['STAP_sigmab_smth', 'STAP_sigmag_smth', 'STAP_sigmar_smth']
    stap_colors = ['blue', 'green', 'red']
    stap_labels = ['STAP_blue', 'STAP_green', 'STAP_red']
    
    for var, color, label in zip(stap_vars, stap_colors, stap_labels):
        if var in flight_data.columns and not flight_data[var].isna().all():
            ax.scatter(time_data, flight_data[var], color=color, label=label, alpha=0.7, s=15, marker='s')
            has_data = True
    
    ax.axvline(x=descent_start_time, color='red', linestyle='--', linewidth=2, alpha=0.8)
    
    # Add flag shading
    add_time_shading(ax, time_data, 
                     flag_cloud=flight_data.get('flag_cloud'),
                     flag_hovering=flight_data.get('flag_hovering'),
                     flag_pollution=flight_data.get('flag_pollution'))
    
    if not has_data:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
    else:
        ax.legend(fontsize=6, ncol=2)
        ax.grid(True, alpha=0.3)
    
    ax.set_title('Microaethalometer/STAP')
    ax.set_ylabel('Abs. Coeff / σ')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
    ax.set_xlim(time_extent)    
    
    # Set x-axis label for bottom panel
    ax.set_xlabel('Time')
    
    plt.tight_layout()

    # Save the figure
    output_file = f"Flight_{flight}_{flight_date}_TimeSeries_Level2.png"
    output_filename = os.path.join(output_dir, output_file)

    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    print(f"Saved figure: {output_filename}")

    plt.show()
    plt.close(fig)
    
    print(f"Completed flight {flight}")
    
print("All flights processed!")

# Average profile for the entire campaign

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Remember to select the campaign if running only this part

def bin_data_by_altitude(data, altitude_bins):
    """
    Bin data by altitude and calculate mean and standard deviation for each bin
    """
    binned_data = {}
    
    # Create altitude bin centers
    bin_centers = (altitude_bins[:-1] + altitude_bins[1:]) / 2
    
    # Digitize altitudes to bins
    bin_indices = np.digitize(data['Altitude'], altitude_bins)
    
    # Initialize results dictionary
    result = {'altitude': bin_centers}
    
    # Variables to process (using exact column names)
    variables = ['Temperature', 'RH', 'WindSpeed', 'WindDir', 'Partector_total_N', 
                'CPC_total_N', 'mSEMS_total_N', 'POPS_total_N', 'mCDA_total_N', 'CO2',
                'MA_Abs_Coeff_375', 'MA_Abs_Coeff_450', 'MA_Abs_Coeff_528', 
                'MA_Abs_Coeff_625', 'MA_Abs_Coeff_880', 'STAP_sigmab_smth', 
                'STAP_sigmag_smth', 'STAP_sigmar_smth']
    
    for var in variables:
        if var in data.columns:
            means = []
            stds = []
            counts = []
            
            for i in range(1, len(altitude_bins)):
                mask = bin_indices == i
                bin_data = data[mask][var].dropna()
                
                if len(bin_data) > 0:
                    means.append(bin_data.mean())
                    stds.append(bin_data.std() if len(bin_data) > 1 else 0)
                    counts.append(len(bin_data))
                else:
                    means.append(np.nan)
                    stds.append(np.nan)
                    counts.append(0)
            
            result[f'{var}_mean'] = np.array(means)
            result[f'{var}_std'] = np.array(stds)
            result[f'{var}_count'] = np.array(counts)
    
    return result

def add_altitude_shading_cumulative(ax, altitude_range):
    """Add general altitude shading for cumulative plots"""
    xlim = ax.get_xlim()
    # Add some general atmospheric layers if desired
    # This is optional and can be customized based on your needs
    pass

# Define altitude bins 
# altitude_bins = np.arange(0, 625, 25) # every 25m from 0 to 600m
altitude_bins = np.arange(0, 610, 10) # every 10m from 0 to 600m

print("Processing all flights for cumulative analysis...")

# Filter out rows with missing altitude data
valid_data = df[df['Altitude'].notna() & (df['Altitude'] >= 0) & (df['Altitude'] <= 600)].copy()

if valid_data.empty:
    print("No valid altitude data found!")
    exit()

print(f"Processing {len(valid_data)} data points across all flights")

# Bin all data by altitude
binned_result = bin_data_by_altitude(valid_data, altitude_bins)

# Create figure with subplots (1 row, 8 columns)
fig, axes = plt.subplots(1, 8, figsize=(24, 8))
fig.suptitle('Cumulative Flight Analysis - Vertical Profiles (Level 2)', fontsize=16, fontweight='bold')

# Adjust layout
plt.subplots_adjust(wspace=0.3)

# Set y-axis limits for all plots
ylim = [0, 600]
altitude = binned_result['altitude']

# Panel 1: Flight Count vs Altitude
ax = axes[0]
total_counts = np.zeros_like(altitude)
variables = ['Temperature', 'RH', 'WindSpeed', 'Partector_total_N', 'CPC_total_N', 
            'mSEMS_total_N', 'POPS_total_N', 'CO2']

# Calculate average data availability across key variables
for var in variables:
    if f'{var}_count' in binned_result:
        total_counts += binned_result[f'{var}_count']

avg_counts = total_counts / len(variables)
ax.plot(avg_counts, altitude, 'ko-', markersize=4)
ax.set_xlabel('Average Data Points per Bin')
ax.set_ylabel('Altitude (m)')
ax.set_title('Data Availability')
ax.set_ylim(ylim)
ax.grid(True, alpha=0.3)

# Panel 2: Temperature vs Altitude
ax = axes[1]
if 'Temperature_mean' in binned_result:
    temp_mean = binned_result['Temperature_mean']
    temp_std = binned_result['Temperature_std']
    
    # Plot mean with error bars
    valid_mask = ~np.isnan(temp_mean)
    if np.any(valid_mask):
        ax.errorbar(temp_mean[valid_mask], altitude[valid_mask], 
                   xerr=temp_std[valid_mask], fmt='r-', capsize=3, alpha=0.7)
        ax.plot(temp_mean[valid_mask], altitude[valid_mask], 'ro', markersize=4)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')

ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Altitude (m)')
ax.set_title('Temperature')
ax.set_ylim(ylim)

# Panel 3: Relative Humidity vs Altitude
ax = axes[2]
if 'RH_mean' in binned_result:
    rh_mean = binned_result['RH_mean']
    rh_std = binned_result['RH_std']
    
    valid_mask = ~np.isnan(rh_mean)
    if np.any(valid_mask):
        ax.errorbar(rh_mean[valid_mask], altitude[valid_mask], 
                   xerr=rh_std[valid_mask], fmt='b-', capsize=3, alpha=0.7)
        ax.plot(rh_mean[valid_mask], altitude[valid_mask], 'bo', markersize=4)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')

ax.set_xlabel('Relative Humidity (%)')
ax.set_ylabel('Altitude (m)')
ax.set_title('Humidity')
ax.set_ylim(ylim)

# Panel 4: Wind Speed vs Altitude
ax = axes[3]
ax_right = ax.twiny()

has_wind_speed = 'WindSpeed_mean' in binned_result
has_wind_dir = 'WindDir_mean' in binned_result

if has_wind_speed:
    ws_mean = binned_result['WindSpeed_mean']
    ws_std = binned_result['WindSpeed_std']
    
    valid_mask = ~np.isnan(ws_mean)
    if np.any(valid_mask):
        ax.errorbar(ws_mean[valid_mask], altitude[valid_mask], 
                   xerr=ws_std[valid_mask], fmt='g-', capsize=3, alpha=0.7)
        ax.plot(ws_mean[valid_mask], altitude[valid_mask], 'go', markersize=4)
        ax.grid(True, alpha=0.3)

if has_wind_dir:
    wd_mean = binned_result['WindDir_mean']
    wd_std = binned_result['WindDir_std']
    
    valid_mask = ~np.isnan(wd_mean)
    if np.any(valid_mask):
        ax_right.errorbar(wd_mean[valid_mask], altitude[valid_mask], 
                         xerr=wd_std[valid_mask], fmt='r-', capsize=2, alpha=0.5)
        ax_right.scatter(wd_mean[valid_mask], altitude[valid_mask], 
                        c='red', s=20, alpha=0.7)
        ax_right.set_xlim(0, 360)
        ax_right.set_xlabel('Wind Direction (°)', color='red')
        ax_right.tick_params(axis='x', labelcolor='red')

if not has_wind_speed and not has_wind_dir:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')

ax.set_xlabel('Wind Speed (m/s)', color='green')
ax.set_ylabel('Altitude (m)')
ax.set_title('Wind')
ax.set_ylim(ylim)
ax.tick_params(axis='x', labelcolor='green')

# Panel 5: Particle Counters vs Altitude
ax = axes[4]

# Plot different particle counters
colors = ['blue', 'green', 'red']
labels = ['Partector', 'CPC', 'mSEMS']
variables = ['Partector_total_N', 'CPC_total_N', 'mSEMS_total_N']

has_data = False

for var, color, label in zip(variables, colors, labels):
    if f'{var}_mean' in binned_result:
        mean_data = binned_result[f'{var}_mean']
        std_data = binned_result[f'{var}_std']
        
        valid_mask = ~np.isnan(mean_data)
        if np.any(valid_mask):
            ax.errorbar(mean_data[valid_mask], altitude[valid_mask], 
                       xerr=std_data[valid_mask], fmt=f'{color[0]}-', 
                       capsize=2, alpha=0.7, label=label)
            has_data = True

if not has_data:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

ax.set_xlabel('Particle Number (#/cm³)')
ax.set_ylabel('Altitude (m)')
ax.set_title('Particles')
ax.set_ylim(ylim)

# Panel 6: POPS and miniCDA Total vs Altitude
ax = axes[5]

has_data = False

if 'POPS_total_N_mean' in binned_result:
    pops_mean = binned_result['POPS_total_N_mean']
    pops_std = binned_result['POPS_total_N_std']
    
    valid_mask = ~np.isnan(pops_mean)
    if np.any(valid_mask):
        ax.errorbar(pops_mean[valid_mask], altitude[valid_mask], 
                   xerr=pops_std[valid_mask], fmt='o-', color='purple', 
                   capsize=3, alpha=0.7, label='POPS', markersize=4)
        has_data = True

if 'mCDA_total_N_mean' in binned_result:
    mcda_mean = binned_result['mCDA_total_N_mean']
    mcda_std = binned_result['mCDA_total_N_std']
    
    valid_mask = ~np.isnan(mcda_mean)
    if np.any(valid_mask):
        ax.errorbar(mcda_mean[valid_mask], altitude[valid_mask], 
                   xerr=mcda_std[valid_mask], fmt='o-', color='orange', 
                   capsize=3, alpha=0.7, label='miniCDA', markersize=4)
        has_data = True

if not has_data:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

ax.set_xlabel('Total N (#/cm³)')
ax.set_ylabel('Altitude (m)')
ax.set_title('POPS & miniCDA')
ax.set_ylim(ylim)

# Panel 7: CO2 vs Altitude
ax = axes[6]
if 'CO2_mean' in binned_result:
    co2_mean = binned_result['CO2_mean']
    co2_std = binned_result['CO2_std']
    
    valid_mask = ~np.isnan(co2_mean)
    if np.any(valid_mask):
        ax.errorbar(co2_mean[valid_mask], altitude[valid_mask], 
                   xerr=co2_std[valid_mask], fmt='o-', color='brown', 
                   capsize=3, alpha=0.7, markersize=4)
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')

ax.set_xlabel('CO2 (ppm)')
ax.set_ylabel('Altitude (m)')
ax.set_title('CO2')
ax.set_ylim(ylim)

# Panel 8: STAP Sigma Values or Microaethalometer Absorption Coefficients vs Altitude
ax = axes[7]

has_data = False

# Check for microaethalometer data first
microaeth_vars = ['MA_Abs_Coeff_375', 'MA_Abs_Coeff_450', 'MA_Abs_Coeff_528', 
                  'MA_Abs_Coeff_625', 'MA_Abs_Coeff_880']
microaeth_colors = ['purple', 'blue', 'green', 'red', 'black']
microaeth_labels = ['375nm', '450nm', '528nm', '625nm', '880nm']

has_microaeth = any(f'{var}_mean' in binned_result and 
                   not np.isnan(binned_result[f'{var}_mean']).all() 
                   for var in microaeth_vars)

if has_microaeth:
    # Plot microaethalometer data
    for var, color, label in zip(microaeth_vars, microaeth_colors, microaeth_labels):
        if f'{var}_mean' in binned_result:
            mean_data = binned_result[f'{var}_mean']
            std_data = binned_result[f'{var}_std']
            
            valid_mask = ~np.isnan(mean_data)
            if np.any(valid_mask):
                ax.errorbar(mean_data[valid_mask], altitude[valid_mask], 
                           xerr=std_data[valid_mask], fmt='o-', color=color, 
                           capsize=2, alpha=0.7, label=label, markersize=3)
                has_data = True
    
    ax.set_xlabel('Absorption Coefficient')
    ax.set_title('Microaethalometer')
else:
    # Plot STAP data
    stap_vars = ['STAP_sigmab_smth', 'STAP_sigmag_smth', 'STAP_sigmar_smth']
    stap_colors = ['blue', 'green', 'red']
    stap_labels = ['σ_blue', 'σ_green', 'σ_red']
    
    for var, color, label in zip(stap_vars, stap_colors, stap_labels):
        if f'{var}_mean' in binned_result:
            mean_data = binned_result[f'{var}_mean']
            std_data = binned_result[f'{var}_std']
            
            valid_mask = ~np.isnan(mean_data)
            if np.any(valid_mask):
                ax.errorbar(mean_data[valid_mask], altitude[valid_mask], 
                           xerr=std_data[valid_mask], fmt='o-', color=color, 
                           capsize=2, alpha=0.7, label=label, markersize=3)
                has_data = True
    
    ax.set_xlabel('STAP σ')
    ax.set_title('STAP')

if not has_data:
    ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center', va='center')
else:
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

ax.set_ylabel('Altitude (m)')
ax.set_ylim(ylim)

plt.tight_layout()

# Save the figure
campaign_name = valid_data['campaign'].iloc[0] if 'campaign' in valid_data.columns else 'Unknown_Campaign'
output_file = f"Flights_{campaign_name}_Average_Profile_Level2.png"
output_filename = os.path.join(output_dir, output_file)

plt.savefig(output_filename, dpi=300, bbox_inches='tight')
print(f"Saved average profile figure: {output_filename}")

plt.show()

# Print some statistics
print("\nAverage Profile Summary:")
print(f"Total data points processed: {len(valid_data)}")
print(f"Altitude range: {valid_data['Altitude'].min():.1f} - {valid_data['Altitude'].max():.1f} m")
print(f"Number of flights: {valid_data['flight_nr'].nunique()}")
print(f"Date range: {valid_data['DateTime'].min().date()} to {valid_data['DateTime'].max().date()}")

print("Average profile completed!")

# Overview of the entire campaign

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Remember to select the campaign if running only this part
print("Processing all flights for the campaign overview profile...")

# Get unique flights and sort by date
flights = df['flight_nr'].unique()
flight_dates = []
for flight in flights:
    flight_data = df[df['flight_nr'] == flight]
    if not flight_data.empty and 'DateTime' in flight_data.columns:
        flight_dates.append((flight, flight_data['DateTime'].min()))

# Sort flights by date
flight_dates.sort(key=lambda x: x[1])
sorted_flights = [f[0] for f in flight_dates]

# Create colormap - viridis from blue (oldest) to yellow (newest)
colors = plt.cm.viridis(np.linspace(0, 1, len(sorted_flights)))

# Set y-axis limits
max_alt = df['Altitude'].max()
ylim_max = int(np.ceil(max_alt / 100.0) * 100)
ylim = [0, ylim_max]

# Create figure with subplots (1 row, 7 columns)
fig, axes = plt.subplots(1, 7, figsize=(24, 8))
fig.suptitle('All Flights Overlay - Vertical Profiles (Level 2)', fontsize=16, fontweight='bold', y=0.98)

# Adjust layout to make room for colorbar at top
plt.subplots_adjust(wspace=0.3, top=0.88)

# Process each flight
for flight_idx, flight in enumerate(sorted_flights):
    print(f"Processing flight {flight} ({flight_idx+1}/{len(sorted_flights)})")
    
    flight_data = df[df['flight_nr'] == flight].copy()
    flight_data = flight_data.sort_values('DateTime')
    
    if 'Altitude' not in flight_data.columns or flight_data['Altitude'].isna().all():
        print(f"  No altitude data, skipping...")
        continue
    
    color = colors[flight_idx]
    
    # Panel 0: Temperature vs Altitude
    ax = axes[0]
    if 'Temperature' in flight_data.columns and not flight_data['Temperature'].isna().all():
        ax.plot(flight_data['Temperature'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('Temperature (°C)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('Temperature')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 1: Relative Humidity vs Altitude
    ax = axes[1]
    if 'RH' in flight_data.columns and not flight_data['RH'].isna().all():
        ax.plot(flight_data['RH'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('Relative Humidity (%)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('Humidity')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 2: Wind Speed vs Altitude
    ax = axes[2]
    if 'WindSpeed' in flight_data.columns and not flight_data['WindSpeed'].isna().all():
        ax.plot(flight_data['WindSpeed'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('Wind Speed (m/s)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('Wind Speed')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 3: Particle Counters vs Altitude
    ax = axes[3]
    
    if 'CPC_total_N' in flight_data.columns and not flight_data['CPC_total_N'].isna().all():
        ax.plot(flight_data['CPC_total_N'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('Total N (#/cm³)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('Particles (CPC)')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 4: POPS and miniCDA Total vs Altitude
    ax = axes[4]
    
    if 'POPS_total_N' in flight_data.columns and not flight_data['POPS_total_N'].isna().all():
        ax.plot(flight_data['POPS_total_N'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('Total N (#/cm³)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('POPS')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 5: CO2 vs Altitude
    ax = axes[5]
    
    if 'CO2' in flight_data.columns and not flight_data['CO2'].isna().all():
        ax.plot(flight_data['CO2'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    ax.set_xlabel('CO2 (ppm)')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('CO2')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)
    
    # Panel 6: STAP Sigma Values or Microaethalometer Absorption Coefficients vs Altitude
    ax = axes[6]
    
    # Check for microaethalometer data 
    microaeth_vars = ['MA_Abs_Coeff_375', 'MA_Abs_Coeff_450', 'MA_Abs_Coeff_528', 'MA_Abs_Coeff_625', 'MA_Abs_Coeff_880']
    
    has_microaeth = any(var in flight_data.columns and not flight_data[var].isna().all() for var in microaeth_vars)
    
    if has_microaeth:
        # Plot microaethalometer data (use one wavelength for simplicity, e.g., 880nm)
        if 'MA_Abs_Coeff_880' in flight_data.columns and not flight_data['MA_Abs_Coeff_880'].isna().all():
            ax.plot(flight_data['MA_Abs_Coeff_880'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    else:
        # Plot STAP data (use one channel for simplicity, e.g., red)
        if 'STAP_sigmar_smth' in flight_data.columns and not flight_data['STAP_sigmar_smth'].isna().all():
            ax.plot(flight_data['STAP_sigmar_smth'], flight_data['Altitude'], '-', color=color, alpha=0.6, linewidth=1.5)
    
    ax.set_xlabel('Abs. Coeff / σ')
    ax.set_ylabel('Altitude (m)')
    ax.set_title('mAETH/STAP')
    ax.set_ylim(ylim)
    ax.grid(True, alpha=0.3)

# Add colorbar to show flight progression (at top, between title and panels)
cbar_ax = fig.add_axes([0.15, 0.92, 0.7, 0.02])  # [left, bottom, width, height]
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=len(sorted_flights)-1))
sm.set_array([])
cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Flight Number', fontsize=11)
cbar.set_ticks(np.arange(0, len(sorted_flights), max(1, len(sorted_flights)//10)))
cbar.set_ticklabels([sorted_flights[i] for i in range(0, len(sorted_flights), max(1, len(sorted_flights)//10))])

plt.tight_layout(rect=[0, 0, 1, 0.88])

# Save the figure
campaign_name = df['campaign'].iloc[0] if 'campaign' in df.columns else 'Unknown_Campaign'
output_file = f"Flights_{campaign_name}_Overview_Profile_Level2.png"
output_filename = os.path.join(output_dir, output_file)

plt.savefig(output_filename, dpi=300, bbox_inches='tight')
print(f"Saved overview figure: {output_filename}")

plt.show()

# Print summary
print("\nOverview Profile Summary:")
print(f"Total flights plotted: {len(sorted_flights)}")
print(f"Flight date range: {flight_dates[0][1].date()} to {flight_dates[-1][1].date()}")

print("Overlay profile completed!")